# Project 1: Digit Classification with KNN and Naive Bayes

In this project, you'll implement your own image recognition system for classifying digits. Read through the code and the instructions carefully and add your own code where indicated. Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but <b> please prepare your own write-up (with your own code). </b>

If you're interested, check out these links related to digit recognition:

* Yann Lecun's MNIST benchmarks: http://yann.lecun.com/exdb/mnist/
* Stanford Streetview research and data: http://ufldl.stanford.edu/housenumbers/

Finally, if you'd like to get started with Tensorflow, you can read through this tutorial: https://www.tensorflow.org/tutorials/keras/basic_classification. It uses a dataset called "fashion_mnist", which is identical in structure to the original digit mnist, but uses images of clothing rather than images of digits. The number of training examples and number of labels is the same. In fact, you can simply replace the code that loads "fashion_mnist" with "mnist" and everything should work fine.

In [ ]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# Import a bunch of libraries.
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Set the randomizer seed so results are the same each time.
np.random.seed(0)

In [ ]:
import sklearn
sklearn.__version__

Load the data. Notice that the data gets partitioned into training, development, and test sets. Also, a small subset of the training data called mini_train_data and mini_train_labels gets defined, which you should use in all the experiments below, unless otherwise noted.

In [ ]:
# Load the digit data from https://www.openml.org/d/554 or from default local location '~/scikit_learn_data/...'
X, Y = fetch_openml(name='mnist_784', return_X_y=True, cache=False)


# Rescale grayscale values to [0,1].
X = X / 255.0

# Shuffle the input: create a random permutation of the integers between 0 and the number of data points and apply this
# permutation to X and Y.
# NOTE: Each time you run this cell, you'll re-shuffle the data, resulting in a different ordering.
shuffle = np.random.permutation(np.arange(X.shape[0]))
X, Y = X[shuffle], Y[shuffle]

print('data shape: ', X.shape)
print('label shape:', Y.shape)

# Set some variables to hold test, dev, and training data.
test_data, test_labels = X[61000:], Y[61000:]
dev_data, dev_labels = X[60000:61000], Y[60000:61000]
train_data, train_labels = X[:60000], Y[:60000]
mini_train_data, mini_train_labels = X[:1000], Y[:1000]

### Part 1:

Show a 10x10 grid that visualizes 10 examples of each digit.

Notes:
* You can use `plt.rc()` for setting the colormap, for example to black and white.
* You can use `plt.subplot()` for creating subplots.
* You can use `plt.imshow()` for rendering a matrix.
* You can use `np.array.reshape()` for reshaping a 1D feature vector into a 2D matrix (for rendering).

In [ ]:
alldigits = ['0','1','2','3','4','5','6','7','8','9']

def P1(num_examples=10):


### STUDENT START ###
    numrow = 1
    numcol = num_examples

##### TEST ###   
#numrow = 2
#numcol = 2
#fig, axes = plt.subplots(2,2) 
#pic[0]
#axes[0,0].imshow(example[0].reshape(28,28))
#axes[0,1].imshow(example[1].reshape(28,28))
#plt.show()

    fig, axes = plt.subplots(numrow,numcol) 
    for i in range(numcol):
        axes[i].imshow(X0[i,].reshape(28,28))
        axes[i].xaxis.set_visible(False)
        axes[i].yaxis.set_visible(False)
    plt.show()
### STUDENT END ###

for digit in alldigits:
    X0 = mini_train_data[mini_train_labels == digit]
#    X0[:10,]
    P1(10)


### Part 2:

Produce k-Nearest Neighbors models with k $\in$ [1,3,5,7,9].  Evaluate and show the accuracy of each model. For the 1-Nearest Neighbor model, additionally show the precision, recall, and F1 for each label. Which digit is the most difficult for the 1-Nearest Neighbor model to recognize?

Notes:
* Train on the mini train set.
* Evaluate performance on the dev set.
* You can use `KNeighborsClassifier` to produce a k-nearest neighbor model.
* You can use `classification_report` to get precision, recall, and F1 results.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric

def P2(k_values):


### STUDENT START ###
    for i in k_values:
        model = KNeighborsClassifier(n_neighbors=i)
        model.fit(mini_train_data, mini_train_labels)
        dev_predicted_labels = model.predict(dev_data)
        out = classification_report(dev_labels, dev_predicted_labels, output_dict=True)
        print("k=%d: Accuracy ... " % i,out['accuracy'])
        if (i == 1):
#           extract all f1-scores
            scores = []
            for j in alldigits:
                scores.append(out[j]['f1-score'])
                print('f1-score for ',j, out[j]['f1-score'])
#           find index(es) of minimum 
            indexes = [k for k, x in enumerate(scores) if x == min(scores)]
            print("     Most difficult digit(s): ", indexes) 
### STUDENT END ###

k_values = [1, 3, 5, 7, 9]
P2(k_values)

ANSWER: The answer is the one with the lowest f1-score, the digit 8. 

### Part 3:

Produce 1-Nearest Neighbor models using training data of various sizes.  Evaluate and show the performance of each model.  Additionally, show the time needed to measure the performance of each model.

Notes:
* Train on subsets of the train set.  For each subset, take just the first part of the train set without re-ordering.
* Evaluate on the dev set.
* You can use `KNeighborsClassifier` to produce a k-nearest neighbor model.
* You can use `time.time()` to measure elapsed time of operations.

In [ ]:
import time 
def P3(train_sizes, accuracies):

### STUDENT START ###
    k=1
    for s in train_sizes:
        model = KNeighborsClassifier(n_neighbors=k)#, algorithm='kd_tree', leaf_size=1000)
        traindata = train_data[:s]
        trainlabels = train_labels[:s]
        model.fit(traindata, trainlabels)
        time0 = time.time()
        dev_predicted_labels = model.predict(dev_data)
        time1 = time.time() - time0
        out = classification_report(dev_labels, dev_predicted_labels, output_dict=True)
        acc = out["accuracy"]
        accuracies.append(acc)
        print("train size %d: " % s, "\n elapsed time {:.3f} , Accuracy {:.3f} ".format(time1,acc))
### STUDENT END ###

train_sizes = [100, 200, 400, 800, 1600, 3200, 6400, 12800, 25600]
#train_sizes = [100,200, 400, 800, 1600, 3200]
#train_sizes = [100]
accuracies = []
P3(train_sizes, accuracies)
#print("accuracies: %5.3f"% (accuracies))
print('\nAccuracies')
print(', '.join('{:.3f}'.format(f) for f in accuracies))


The times displayed are for model.predict(traindata)

### Part 4:

Produce a linear regression model that predicts accuracy of a 1-Nearest Neighbor model given training set size. Show $R^2$ of the linear regression model.  Show the accuracies predicted for training set sizes 60000, 120000, and 1000000.  Show a lineplot of actual accuracies and predicted accuracies vs. training set size over the range of training set sizes in the training data.  What's wrong with using linear regression here?

Apply a transformation to the predictor features and a transformation to the outcome that make the predictions more reasonable.  Show $R^2$ of the improved linear regression model.  Show the accuracies predicted for training set sizes 60000, 120000, and 1000000.  Show a lineplot of actual accuracies and predicted accuracies vs. training set size over the range of training set sizes in the training data - be sure to display accuracies and training set sizes in appropriate units.

Notes:
* Train the linear regression models on all of the (transformed) accuracies estimated in Problem 3.
* Evaluate the linear regression models on all of the (transformed) accuracies estimated in Problem 3.
* You can use `LinearRegression` to produce a linear regression model.
* Remember that the sklearn `fit()` functions take an input matrix X and output vector Y. So, each input example in X is a vector, even if it contains only a single value.
* Hint re: predictor feature transform: Accuracy increases with training set size logarithmically.
* Hint re: outcome transform: When y is a number in range 0 to 1, then odds(y)=y/(1-y) is a number in range 0 to infinity.

In [ ]:
from scipy import stats
def P4():

### STUDENT START ###
    sizes=np.array(train_sizes).reshape(-1,1)
    linreg = LinearRegression(fit_intercept = True, copy_X = True).fit(sizes,accuracies)
    pred=linreg.predict(np.array(sizes))
    #p.show

    fig, ax = plt.subplots()

    ax.scatter(sizes, accuracies, c='blue', label="actual accuracies")
    ax.plot(sizes, pred, c='green', label="fitted accuracies")
    ax.legend()
    ax.grid(True)
    ax.title.set_text('accuracies = a.sizes + b')

    odds_accuracies = np.array(accuracies) / (1-np.array(accuracies))

    loglinreg = LinearRegression(fit_intercept = True, copy_X = True).fit(np.log(sizes),odds_accuracies)
    logpred=loglinreg.predict(np.array(np.log(sizes)))
    fig1, ax1 = plt.subplots()
    ax1.scatter(np.log(sizes), odds_accuracies, c='red', label="odds(actual accuracies) versus ln(sizes)")
    ax1.plot(np.log(sizes), logpred, c='green', label="fitted odds(accuracies) as function of ln(sizes)")
    ax1.legend()
    ax1.grid(True)
    ax1.title.set_text('odds(accuracies) = a.ln(sizes) + b')

    fig.show()
    fig1.show()
    print("Linear Regression r squared: ",linreg.score(sizes,accuracies))
    print("Log linear Regression r squared: ",loglinreg.score(np.log(sizes),odds_accuracies))

    larger = [60000,120000,1000000]
    newsizes=np.array(larger).reshape(-1,1)
    newpred = loglinreg.predict(np.array(np.log(newsizes)))
    newpred = newpred/(1+newpred)
    print("Hypothetical training sizes: ",larger)
    print("Their predicted accuracies", newpred)
    print("Their predicted accuracies")
    print(', '.join('{:.3f}'.format(f) for f in newpred))
### STUDENT END ###

P4()

ANSWER:For training sizes 60,000, 120,000 and 1,000,000 the regression gives accuracies of 0.981, 0.983 and 0.987 respectively. Note that: 

1- for a training size of 60,000 the fitted accuracy is lower than the actual accuracy for training size = 25,600. That is because, as can be seen on graph above, the regression underestimates the accuracy for larger training sizes. 

2- As can be seen in top graph above, as training size increases the relative gain in accuracy goes down (negative second derivative) indicating that at some point increasing training size can be counterproductive. That limit must depend on the accuracy reqired by the problem.  

### Part 5:

Produce a 1-Nearest Neighbor model and show the confusion matrix. Which pair of digits does the model confuse most often? Show the images of these most often confused digits.

Notes:
- Train on the mini train set.
- Evaluate performance on the dev set.
- You can use `confusion_matrix()` to produce a confusion matrix.

In [ ]:
def P5():

### STUDENT START ###
        # confusion matrix
        model = KNeighborsClassifier(n_neighbors=1)
        model.fit(mini_train_data, mini_train_labels)
        dev_predicted_labels = model.predict(dev_data)
        out = classification_report(dev_labels, dev_predicted_labels, output_dict=True)
        acc = out["accuracy"]
        cm = confusion_matrix(dev_labels, dev_predicted_labels)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = alldigits)
        disp.plot()

        #find most confused digits
        X0 = cm
        for i in range(10):
            X0[i,i] = 0
            for j in range(i,10):
                X0[i,j] = cm[i,j]+cm[j,i]
                X0[j,i] = 0
        #print(X0)
        #find index(es) of maximum 
        ind = np.unravel_index(np.argmax(X0, axis=None), X0.shape) 
        print("Most confused digits: ", ind) 
        num_most_confused = np.max(X0, axis=None)
        
        #print most confused digits
        d0 = alldigits[ind[0]]
        d1 = alldigits[ind[1]]
        fig, axes = plt.subplots(num_most_confused,2) 
        fig.set_size_inches(5,10)

        j = 0
        i = 0
        while (i < dev_labels.shape[0]):
            if dev_labels[i] == d0:
                if dev_predicted_labels[i] == d1:
                    axes[j,0].imshow(dev_data[i].reshape(28,28))
                    axes[j,0].xaxis.set_visible(False)
                    axes[j,0].yaxis.set_visible(False)
                    axes[j,1].text(-0.15, 0.5, 'Predicted:'+d1,  
                                   fontsize = 12, fontweight = 'bold', 
                                   horizontalalignment='center', verticalalignment='center', transform=axes[j,1].transAxes)
                    axes[j,1].xaxis.set_visible(False)
                    axes[j,1].yaxis.set_visible(False)
                    axes[j,1].axis('off')
                    j += 1
            elif dev_labels[i] == d1:        
                if dev_predicted_labels[i] == d0:
                    axes[j,0].imshow(dev_data[i].reshape(28,28))
                    axes[j,0].xaxis.set_visible(False)
                    axes[j,0].yaxis.set_visible(False)
                    axes[j,1].text(-0.15, 0.5, 'Predicted:'+d0,fontsize = 12, fontweight = 'bold',
                    horizontalalignment='center', verticalalignment='center', transform=axes[j,1].transAxes)
                    axes[j,1].xaxis.set_visible(False)
                    axes[j,1].yaxis.set_visible(False)
                    axes[j,1].axis('off')
                    j += 1
            i += 1
        plt.show()
    
### STUDENT END ###

P5()



ANSWER: this answer is for the largest instances of **pairs** of numbers that are confused. 
 In this case, **both 4 predicted as 9, and  9 predicted as 4.**
Note that for the image before last even I cannot tell whether it is a 4 or a 9!

### Part 6:

A common image processing technique is to smooth an image by blurring. The idea is that the value of a particular pixel is estimated as the weighted combination of the original value and the values around it. Typically, the blurring is Gaussian, i.e., the weight of a pixel's influence is determined by a Gaussian function over the distance to the relevant pixel.

Implement a simplified Gaussian blur filter by just using the 8 neighboring pixels like this: the smoothed value of a pixel is a weighted combination of the original value and the 8 neighboring values.

Pick a weight, then produce and evaluate four 1-Nearest Neighbor models by applying your blur filter in these ways:
- Do not use the filter
- Filter the training data but not the dev data
- Filter the dev data but not the training data
- Filter both training data and dev data

Show the accuracies of the four models evaluated as described.  Try to pick a weight that makes one model's accuracy at least 0.9.

Notes:
* Train on the (filtered) mini train set.
* Evaluate performance on the (filtered) dev set.
* There are other Guassian blur filters available, for example in `scipy.ndimage.filters`. You are welcome to experiment with those, but you are likely to get the best results with the simplified version described above.

In [ ]:
def blur(weight, dataset, newset):
### filters 8 closest neighbors using constant weight ###
    (n,m) = dataset.shape
    # print (n,m)

    image=np.zeros((30,30))
    newimage= np.zeros((28,28))
    # reshape each row of dataset into a square, embed in a larger square image so that 
    # 1st and last columns are zeros and first and last row are zeros
    # this allows to use the same formula for all cells in the image and not worry about cells on the contour
    # each row of dataset has 28x28 = 784 elements
    
    for i in range(0,n):
        image[1:29,1:29] = dataset[i,].reshape(28,28)
        for r in range(1,29):
            for c in range(1,29):
#                print("r ",r," c ",c)
                newimage[r-1,c-1] = ( image[r,c] + 
                    weight * (image[r-1,c-1] + image[r-1,c] + image[r-1,c+1] + 
                       image[r,c-1]+ image[r,c+1] + 
                       image[r+1,c-1] + image[r+1,c] +image[r+1,c+1])) /(1+8*weight)
        newset[i,:] = newimage.reshape(1,784)
      

In [ ]:
def plot_digits(dataset, numcol):
### draw numrows rows of numcol digits
    ## dataset is a numpy array with 28x28 = 784 columns
    ## plot first numcol rows of dataset (rows of dataset become columns of the output)
    ## dataset is a numpy array with numcol rows and 28x28 = 784 columns
    numcol = dataset.shape[0]
    numrow = 1
    fig, axes = plt.subplots(numrow,numcol) 
    for i in range(numcol):
        axes[i].imshow(dataset[i,].reshape(28,28))
        axes[i].xaxis.set_visible(False)
        axes[i].yaxis.set_visible(False)
    plt.show()
#

In [ ]:
def P6():
    
### STUDENT START ###

    model = KNeighborsClassifier(n_neighbors=1)

    # no filter
    model.fit(mini_train_data, mini_train_labels)
    dev_predicted_labels = model.predict(dev_data)
    nofilter_out = classification_report(dev_labels, dev_predicted_labels, output_dict=True)
    print("Not filtered: Accuracy ... ",nofilter_out['accuracy'])
    #extract all f1-scores
#    scores = []
#    for j in alldigits:
#        scores.append(nofilter_out[j]['f1-score'])
        #find index(es) of minimum 
#        indexes = [k for k, x in enumerate(scores) if x == min(scores)]
        # index is same as digit
#        print("     Most difficult digit(s): ", indexes) 

    weight = 1/10

    # filter training set 
    new_train_data = np.ndarray(mini_train_data.shape) 
    blur(weight,mini_train_data,new_train_data)
#    plot_digits(mini_train_data, 10)
#    plot_digits(new_train_data, 10)
    model.fit(new_train_data, mini_train_labels)
    dev_predicted_labels = model.predict(dev_data)
    out = classification_report(dev_labels, dev_predicted_labels, output_dict=True)
    print("Filtered training set: Accuracy ... ",out['accuracy'])
    #extract all f1-scores
#    scores = []
#    for j in alldigits:
#        scores.append(out[j]['f1-score'])
        #find index(es) of minimum 
#        indexes = [k for k, x in enumerate(scores) if x == min(scores)]
#        print("     Most difficult digit(s): ", indexes) 

    weight = 1/10
    # filter dev data
    model.fit(mini_train_data, mini_train_labels)
    new_dev_data = np.ndarray(dev_data.shape) 
    blur(weight, dev_data, new_dev_data)
#    plot_digits(dev_data, 10)
#    plot_digits(new_dev_data, 10)
    dev_predicted_labels = model.predict(new_dev_data)
    out = classification_report(dev_labels, dev_predicted_labels, output_dict=True)
    print("Filtered dev_data: Accuracy ... ",out['accuracy'])
    #   extract all f1-scores
 #   scores = []
 #   for j in alldigits:
 #       scores.append(out[j]['f1-score'])
        #       find index(es) of minimum 
#        indexes = [k for k, x in enumerate(scores) if x == min(scores)]
#        print("     Most difficult digit(s): ", indexes) 
    
    # filter both training and dev data 
    model.fit(new_train_data, mini_train_labels)
    dev_predicted_labels = model.predict(new_dev_data)
    out = classification_report(dev_labels, dev_predicted_labels, output_dict=True)
    print("Filtered training and dev: Accuracy ... ",out['accuracy'])
    #   extract all f1-scores
#    scores = []
#    for j in alldigits:
#        scores.append(out[j]['f1-score'])
        #       find index(es) of minimum, use the fact that the data is same indexes 
#        indexes = [k for k, x in enumerate(scores) if x == min(scores)]
#        print("     Most difficult digit(s): ", indexes) 
    
### STUDENT END ###

P6()

ANSWER: Results above were obtained using a weight of 1/10. 

### Part 7:

Produce two Naive Bayes models and evaluate their performances.  Recall that Naive Bayes estimates P(feature|label), where each label is a categorical, not a real number.

For the first model, map pixel values to either 0 or 1, representing white or black - you should pre-process the data or use `BernoulliNB`'s `binarize` parameter to set the white/black separation threshold to 0.1.  Use `BernoulliNB` to produce the model.

For the second model, map pixel values to either 0, 1, or 2, representing white, gray, or black - you should pre-process the data, seting the white/gray/black separation thresholds to 0.1 and 0.9.  Use `MultinomialNB` to produce the model. 

Show the Bernoulli model accuracy and the Multinomial model accuracy.

Notes:
* Train on the mini train set.
* Evaluate performance on the dev set.
* `sklearn`'s Naive Bayes methods can handle real numbers, but for this exercise explicitly do the mapping to categoricals. 

Does the multinomial version improve the results? Why or why not?

In [ ]:
def P7():

### STUDENT START ###
    model1 = BernoulliNB(alpha=1, binarize=0.1)
    model1.fit(mini_train_data, mini_train_labels)
    dev_predicted_model1 = model1.predict(dev_data)
    model1_out = classification_report(dev_labels, dev_predicted_model1,output_dict=True)
    #print(model1_out)
    print("Binomial Accuracy ... ",model1_out['accuracy'])

    #model 2 
    #build trinomial data set
    trinom_data = np.ones(mini_train_data.shape)
    trinom_data[mini_train_data < 0.1] = 0
    trinom_data[mini_train_data > 0.9] = 2
    # create model2
    model2 = MultinomialNB(alpha=1)
    model2.fit(trinom_data, mini_train_labels)
    dev_predicted_model2 = model2.predict(dev_data)
    model2_out = classification_report(dev_labels, dev_predicted_model2,output_dict=True)
    #print(model2_out)
    print("Multinomial Accuracy ... ",model2_out['accuracy'])
### STUDENT END ###

P7()


ANSWER: The multinomial does not help. The reason is that the original data is or should be binomial (black or white). 
    So there is no good reason to distinguish a third color. 

### Part 8:

Search across several values of the LaPlace smoothing parameter (alpha) to find its effect on a Bernoulli Naive Bayes model's performance.  Show the accuracy at each alpha value.

Notes:
* Set binarization threshold to 0.
* Train on the mini train set.
* Evaluate performance by 5-fold cross-validation. 
* Use `GridSearchCV(..., ..., cv=..., scoring='accuracy', iid=False)` to vary alpha and evaluate performance by cross-validation.
* Cross-validation is based on partitions of the training data, so results will be a bit different than if you had used the dev set to evaluate performance.

What is the best value for alpha? What is the accuracy when alpha is near 0? Is this what you'd expect?

In [ ]:
#import pandas as pd
def P8(alphas):

### STUDENT START ###
    model = BernoulliNB(binarize=0)
    GSCV = GridSearchCV(model, param_grid = alphas, cv=5,scoring='accuracy',verbose=0)
    GSCV.fit(mini_train_data, mini_train_labels)
#    print(sorted(GSCV.cv_results_.keys()))
#    print(sorted(GSCV.cv_results_))
#    df = pd.DataFrame(GSCV.cv_results_)
#    print(df)
    return GSCV    
### STUDENT END ###

alphas = {'alpha': [1.0e-10, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]}
nb = P8(alphas)
print("Best alpha = ", nb.best_params_)


ANSWER: Alpha = 0.001 gives the best result

### Part 9

Produce a model using Guassian Naive Bayes, which is intended for real-valued features, and evaluate performance. You will notice that it does not work so well. Diagnose the problem and apply a simple fix so that the model accuracy is around the same as for a Bernoulli Naive Bayes model. Show the model accuracy before your fix and the model accuracy after your fix.  Explain your solution.

Notes:
* Train on the mini train set.
* Evaluate performance on the dev set.
* Consider the effects of theta and sigma.  These are stored in the model's `theta_` and `sigma_` attributes.

In [ ]:
def P9():
### GaussianNB; change var_smoothing 
### STUDENT END ###
    model=GaussianNB()
    model.fit(mini_train_data,mini_train_labels)
    dev_predicted_model = model.predict(dev_data)
    model_out = classification_report(dev_labels, dev_predicted_model,output_dict=False)
    print("BEFORE")
    print(model_out)
#    model1 = GaussianNB()
    params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}
    GSCV = GridSearchCV(model, param_grid = params_NB, cv=5,scoring='accuracy',verbose=0)
    GSCV.fit(mini_train_data, mini_train_labels)
#    print(sorted(GSCV.cv_results_.keys()))
#    print(sorted(GSCV.cv_results_))
#    df = pd.DataFrame(GSCV.cv_results_)
#    print(df)
#    print("Best Var Smoothing ", GSCV.best_params_) 
    print("AFTER changing VAR SMOOTHING")
    print("Best Estimator ", GSCV.best_estimator_)
    print("Best Score ", GSCV.best_score_)



### STUDENT END ###

P9()


ANSWER: For Var_smoothing = 0.035 the accuracy is the best: 0.799

In [ ]:
def P9_sigma_theta():
### using sigma and/or theta
### STUDENT END ###
    model=GaussianNB()
    model.fit(mini_train_data,mini_train_labels)
    dev_predicted_model = model.predict(dev_data)
    model_out = classification_report(dev_labels, dev_predicted_model,output_dict=True)
    print("Accuracy before any changes... ",model_out['accuracy'])
#    print(model_out)
#    print(model.sigma_)
    sigma_save = model.sigma_
    theta_save = model.theta_
    
    print("\nChanging SIGMA")
    acc = []
    for i in range(1,11):
        sigma_factor = i * 0.1
        model.sigma_ = model.sigma_ * sigma_factor
        dev_predicted_model = model.predict(dev_data)
        model_out = classification_report(dev_labels, dev_predicted_model,output_dict=True)
#    print("model.sigma_ multiplied by: ", sigma_factor)
        print("sigma times {:.3f}".format(sigma_factor), "; Accuracy ... ",model_out['accuracy'])
        acc.append(model_out['accuracy'])
    ind = acc.index(max(acc))
    sigma_factor = (1+ind)*0.1
    print("Best accuracy obtained for sigma_ multiplied by ",sigma_factor)
#    print(model_out)

    print("\nChanging THETA")
    model.sigma_ = sigma_save
    acc=[]
    for i in range(1,11):
        theta_factor = i * 0.25
        model.theta_ = model.theta_ * theta_factor
        dev_predicted_model = model.predict(dev_data)
        model_out = classification_report(dev_labels, dev_predicted_model,output_dict=True)
        acc.append(model_out['accuracy'])
#    print("model.sigma_ multiplied by: ", sigma_factor)
        print("theta times ",theta_factor, "; Accuracy ... ",model_out['accuracy'])
    ind1 = acc.index(max(acc))
    theta_factor = (1+ind1)*0.25
    print("Best accuracy obtained for theta_ multiplied by ",theta_factor)
    

    
#print("model.theta_ multiplied by: ", theta_factor," model.sigma_ multiplied by: ", sigma_factor)
#    print("adding theta times ",theta_factor, "; Accuracy ... ",model_out['accuracy'])
#    print(model_out)

    model.sigma_ = sigma_save * sigma_factor
    model.theta_ = theta_save * theta_factor
    dev_predicted_model = model.predict(dev_data)
    model_out = classification_report(dev_labels, dev_predicted_model,output_dict=True)
    print("\nChanging SIGMA and THETA")
    print("For model.theta_ multiplied by: ",theta_factor,"; and model.sigma_ multiplied by: ",sigma_factor)
    print("Accuracy ... ",model_out['accuracy'])
#    print(model_out)

### STUDENT END ###

P9_sigma_theta()

In [ ]:
ANSWER:
    
    To improve accuracy I found that the better way was to do a grid search on var_smoothing, but I am not able to 
    explain why. I was not able to find a good explanation of why adjust the variance of one dimension would improve 
    the accuracy.
    
    Changing Sigma and/or Theta: I tried a similar method as the grid search by hand. 
    The best improvement accuracy is when sigma alone is reduced. 
    Larger values of sigma would be equivalent to calculating a weighted average of the feature over a larger area.
    That would diffuse the picture, make it more fuzzy, and therefore harder to decipher 

### Part 10:

Because Naive Bayes produces a generative model, you can use it to generate digit images.

Produce a Bernoulli Naive Bayes model and then use it to generate a 10x20 grid with 20 example images of each digit. Each pixel output should be either 0 or 1, based on comparing some randomly generated number to the estimated probability of the pixel being either 0 or 1.  Show the grid.

Notes:
* You can use np.random.rand() to generate random numbers from a uniform distribution.
* The estimated probability of each pixel being 0 or 1 is stored in the model's `feature_log_prob_` attribute. You can use `np.exp()` to convert a log probability back to a probability.

How do the generated digit images compare to the training digit images?

In [ ]:
def P10(num_examples):
### use predicted joint probability to generate new images   

### STUDENT START ###
    model = BernoulliNB(alpha=1, binarize=0.1)
    model.fit(mini_train_data, mini_train_labels)
    dev_predicted_model = model.predict(dev_data)
    model_out = classification_report(dev_labels, dev_predicted_model,output_dict=True)
    #print(model1_out)
    print("Binomial Accuracy ... ",model_out['accuracy'])

    # get probabilities
    prob = np.exp(model.feature_log_prob_)
#    print(prob.shape)
#    print(prob[1,])

    for digit in range(10):
        pixels = np.ones(shape=(num_examples,784))
        for ex in range(num_examples):
            for i in range(784):
                # sample from prob
                n = np.random.rand()
                if n > prob[digit,i]:
                    pixels[ex,i] = 0
        plot_digits(pixels, num_examples)

### STUDENT END ###

P10(20)

ANSWER: The generated images look worse than the training images. Could it be a coincidence? 
    A sample of size 20 is too small to arrive at a conclusion.

### Part 11:

Recall that a strongly calibrated classifier is rougly 90% accurate when the posterior probability of the predicted class is 0.9. A weakly calibrated classifier is more accurate when the posterior probability of the predicted class is 90% than when it is 80%. A poorly calibrated classifier has no positive correlation between posterior probability and accuracy.  

Produce a Bernoulli Naive Bayes model.  Evaluate performance: partition the dev set into several buckets based on the posterior probabilities of the predicted classes - think of a bin in a histogram- and then estimate the accuracy for each bucket. So, for each prediction, find the bucket to which the maximum posterior probability belongs, and update "correct" and "total" counters accordingly.  Show the accuracy for each bucket.

Notes:
* Set LaPlace smoothing (alpha) to the optimal value (from part 8).
* Set binarization threshold to 0.
* Train on the mini train set.
* Evaluate perfromance on the dev set.

How would you characterize the calibration for this Bernoulli Naive Bayes model?

In [ ]:
def P11(buckets, correct, total):
    
### STUDENT START ###
    model = BernoulliNB(alpha=0.001, binarize=0)
    model.fit(mini_train_data, mini_train_labels)
    dev_predicted_labels = model.predict(dev_data)
    model_out = classification_report(dev_labels, dev_predicted_labels,output_dict=True)
    #print(model1_out)
    print("Binomial Accuracy ... ",model_out['accuracy'])

    # get probabilities
    predicted_prob = model.predict_proba(dev_data)
    # print(predicted_prob.shape)
    
    # print probabilities for first digit in dev data.
    # predicted_proba[0,i]) is the probability of the first number being 'i'
    for i in range(10):
        print("Predicted prob. of digit being ",i,"...", predicted_prob[0,i])
#    print("Actual label: ", dev_labels[0], "  Predicted label:", dev_predicted_labels[0])

    # sort dev_data into buckets based on their predicted probability
    for i in range(len(dev_data)):
        pred=int(dev_predicted_labels[i])
        #prob is the model's predicted probability that dev_data[i] is dev_predicted_label 
        prob = predicted_prob[i][pred]
        # if (i==0): print(prob ," ", i, " ", pred)
            
        # for each bucket count the % of correct predictions
        j=0
        while j < len(buckets):
            if prob > buckets[j]:
                j +=1
            else:
                total[j] += 1
                if dev_predicted_labels[i] == dev_labels[i]:
                    correct[j] += 1
                break
 
#    print(total)
#    print(correct)
### STUDENT END ###

    
buckets = [0.5, 0.9, 0.999, 0.99999, 0.9999999, 0.999999999, 0.99999999999, 0.9999999999999, 1.0]
correct = [0 for i in buckets]
total = [0 for i in buckets]

P11(buckets, correct, total)
#print(total)
#print(correct)

for i in range(len(buckets)):
     accuracy = 0.0
     if ( total[i] > 0): 
        accuracy = correct[i] / total[i]
     print('For bucket %.13f to %.13f    total = %3d    accuracy = %.3f' % (0 if i==0 else buckets[i-1], buckets[i], total[i], accuracy))

ANSWER: It is clear that the accuracy of the model is positively correlated with the predicted probability. 
    However, the accuracy is well below 0.8 for probabilities that are over 90%.  The model is weakly calibrated.

### Part 12 EXTRA CREDIT:

Design new features to see if you can produce a Bernoulli Naive Bayes model with better performance.  Show the accuracy of a model based on the original features and the accuracy of the model based on the new features.

Here are a few ideas to get you started:
- Try summing or averaging the pixel values in each row.
- Try summing or averaging the pixel values in each column.
- Try summing or averaging the pixel values in each square block. (pick various block sizes)
- Try counting the number of enclosed regions. (8 usually has 2 enclosed regions, 9 usually has 1, and 7 usually has 0)

Notes:
* Train on the mini train set (enhanced to comprise the new features).
* Evaulate performance on the dev set.
* Ensure that your code is well commented.

In [ ]:
def P12():

### STUDENT START ###
    
    n = mini_train_data.shape[0] # number of rows of mini_train_set
    x= mini_train_data.reshape(n, 28,28)
    y = np.sum(x,axis=1)  # sum of columns
    z = np.sum(x,axis=2)  # sum of rows
#    print(mini_train_data.shape)
#    print(y.shape)
    new_train_set_1 = np.hstack([mini_train_data,z]) #add sum of rows to mini_train_data 
    new_train_set_2 = np.hstack([mini_train_data,y]) #add sum of columns to mini_train_data
    new_train_set_3 = np.hstack([mini_train_data,z,y])  #add sum of rows and sum of columns to mini_train_data
    
    n = dev_data.shape[0] # number of rows of mini_train_set
    x= dev_data.reshape(n, 28,28)
    y = np.sum(x,axis=1)  # sum of columns
    z = np.sum(x,axis=2)  # sum of rows
#    print(dev_data.shape)
#    print(y.shape)
    new_dev_set_1 = np.hstack([dev_data,z])  #add sum of rows to dev_data 
    new_dev_set_2 = np.hstack([dev_data,y])  #add sum of columns to dev_data
    new_dev_set_3 = np.hstack([dev_data,z,y])  #add sum of rows and sum of columns to dev_data
    
    # original model
    model = BernoulliNB(alpha=1, binarize=0)
    model.fit(mini_train_data, mini_train_labels)
    dev_predicted_model = model.predict(dev_data)
    model_out = classification_report(dev_labels, dev_predicted_model,output_dict=True)
    print("Original accuracy = ",model_out['accuracy'])
    
    # model with sum of rows
    model1 = BernoulliNB(alpha=1, binarize=0)
    model1.fit(new_train_set_1, mini_train_labels)
    dev_predicted_model = model1.predict(new_dev_set_1)
    model1_out = classification_report(dev_labels, dev_predicted_model,output_dict=True)
    print("With sum of rows, accuracy = ",model1_out['accuracy'])
    
    # model with sum of columns
    model2 = BernoulliNB(alpha=1, binarize=0)
    model2.fit(new_train_set_2, mini_train_labels)
    dev_predicted_model = model2.predict(new_dev_set_2)
    model2_out = classification_report(dev_labels, dev_predicted_model,output_dict=True)
    print("With sum of columns, accuracy = ",model2_out['accuracy'])

    # model with sum of columns and sum of rows
    model3 = BernoulliNB(alpha=1, binarize=0)
    model3.fit(new_train_set_3, mini_train_labels)
    dev_predicted_model = model3.predict(new_dev_set_3)
    model3_out = classification_report(dev_labels, dev_predicted_model,output_dict=True)
    print("With sum of rows and columns, accuracy = ",model2_out['accuracy'])
### STUDENT END ###

P12()
